# Manually parameterize an FMU
Tutorial by Johannes Stoljar, Tobias Thummerer

## License

In [ ]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how the manually parameterization of an FMU works if very specific adjustments during system initialization is needed. For this purpose, an IO-FMU model is loaded and the various commands for parameterization are shown on the basis of this model. With this example the user shall be guided how to make certain settings at an FMU. Please note, that parameterization of a simulation is possible in a much easier fashion: Using `fmiSimulate`, `fmiSimulateME` or `fmiSimulateCS` together with a parameter dictionary for the keyword `parameters`.

## Target group
The example is primarily intended for users who work in the field of simulation exchange. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/parameterize.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/parameterize.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/examples/examples/src/parameterize.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |

## Code section

To run the example, the previously installed packages must be included. 

In [ ]:
# imports
using FMI
using FMIZoo

### Simulation setup

Next, the start time and end time of the simulation are set.

In [ ]:
tStart = 0.0
tStop = 1.0
tSave = collect(tStart:tStop)

### Import FMU

In the next lines of code the FMU model from *FMIZoo.jl* is loaded and the information about the FMU is shown.

In [ ]:
# we use an FMU from the FMIZoo.jl
pathToFMU = get_model_filename("IO", "Dymola", "2022x")

myFMU = fmiLoad(pathToFMU)
fmiInfo(myFMU)

### Instantiate and Setup FMU

Next it is necessary to create an instance of the FMU. This is achieved by the command `fmiInstantiate!()`.

In [ ]:
fmiInstantiate!(myFMU; loggingOn=true)

In the following code block, start and end time for the simulation is set by the `fmiSetupExperiment()` command.

In [ ]:
fmiSetupExperiment(myFMU, tStart, tStop)

### Parameterize FMU

In this example, for each data type (`real`, `boolean`, `integer` and `string`) a corresponding input or parameter is selected. From here on, the inputs and parameters will be referred to as parameters for simplicity.

In [ ]:
params = ["p_real", "p_boolean", "p_integer", "p_string"]

At the beginning we want to display the initial state of these parameters, for which the FMU must be in initialization mode. The next function `fmiEnterInitializationMode()` informs the FMU to enter the initialization mode. Before calling this function, the variables can be set. Furthermore, `fmiSetupExperiment()` must be called at least once before calling `fmiEnterInitializationMode()`, in order that the start time is defined.

In [ ]:
fmiEnterInitializationMode(myFMU)

The initial state of these parameters are displayed with the function `fmiGet()`.

In [ ]:
fmiGet(myFMU, params)

The initialization mode is terminated with the function `fmiExitInitializationMode()`. (For the model exchange FMU type, this function switches off all initialization equations, and enters the event mode implicitly.)

In [ ]:
fmiExitInitializationMode(myFMU)

In the next step, a function is defined that generates a random value for each parameter. For the parameter `p_string` a random number is inserted into the string. All parameters are combined to a tuple and output.

In [ ]:
function generateRandomNumbers()
    rndReal = 100 * rand()
    rndBoolean = rand() > 0.5
    rndInteger = round(Integer, 100 * rand())
    rndString = "Random number $(100 * rand())!"

    return rndReal, rndBoolean, rndInteger, rndString
end

The previously defined function is called and the results are displayed in the console.

In [ ]:
paramsVal = generateRandomNumbers()

#### First variant

To show the first variant, it is necessary to terminate and reset the FMU instance. Then, as before, the setup command must be called for the FMU. 

In [ ]:
fmiTerminate(myFMU)
fmiReset(myFMU)
fmiSetupExperiment(myFMU, tStart, tStop)

In the next step it is possible to set the parameters for the FMU. With the first variant it is quickly possible to set all parameters at once. Even different data types can be set with only one command. The command `fmiSet()` selects itself which function is chosen for which data type.  As long as the output of the function gives the status code 0, setting the parameters has worked.

In [ ]:
fmiSet(myFMU, params, collect(paramsVal))

After setting the parameters, it can be checked whether the corresponding parameters were set correctly. For this the function `fmiGet()` can be used as above. To be able to call the function `fmiGet()` the FMU must be in initialization mode.

In [ ]:
fmiEnterInitializationMode(myFMU)
# fmiGet(myFMU, params)
fmiExitInitializationMode(myFMU)

Now the FMU has been initialized correctly, the FMU can be simulated. The `fmiSimulate()` command is used for this purpose. It must be pointed out that the keywords `instantiate=false`, `setup=false` must be set. The keyword `instantiate=false` prevents the simulation command from creating a new FMU instance, otherwise our parameterization will be lost. The keyword `setup=false` prevents the FMU from calling the initialization mode again. The additionally listed keyword `freeInstance=false` prevents that the instance is removed after the simulation. This is only needed in this example, because we want to continue working on the created instance. Another keyword is the `recordValues=parmas[1:3]`, which saves: `p_real`, `p_boolean` and `p_integer` as output. It should be noted that the `fmiSimulate()` function is not capable of outputting string values, so `p_string` is omitted.

In [ ]:
simData = fmiSimulate(myFMU, (tStart, tStop); recordValues=params[1:3], saveat=tSave, 
                        instantiate=false, setup=false, freeInstance=false, terminate=false, reset=false)

#### Second variant

To show the second variant, it is necessary to terminate and reset the FMU instance. Then, as before, the setup command must be called for the FMU. 

In [ ]:
fmiTerminate(myFMU)
fmiReset(myFMU)
fmiSetupExperiment(myFMU, tStart, tStop)

To make sure that the functions work it is necessary to generate random numbers again. As shown already, we call the defined function `generateRandomNumbers()` and output the values.

In [ ]:
rndReal, rndBoolean, rndInteger, rndString = generateRandomNumbers()

In the second variant, the value for each data type is set separately by the corresponding command. By this variant one has the maximum control and can be sure that also the correct data type is set. 

In [ ]:
fmiSetReal(myFMU, "p_real", rndReal)
fmiSetBoolean(myFMU, "p_boolean", rndBoolean)
fmiSetInteger(myFMU, "p_integer", rndInteger)
fmiSetString(myFMU, "p_string", rndString)

To illustrate the functionality of the parameterization with the separate functions, the corresponding get function can be also called separately for each data type:
* `fmiSetReal()` &#8660; `fmiGetReal()`
* `fmiSetBoolean()` &#8660; `fmiGetBoolean()`
* `fmiSetInteger()` &#8660; `fmiGetInteger()`
* `fmiSetString()` &#8660; `fmiGetString()`.

As before, the FMU must be in initialization mode.

In [ ]:
fmiEnterInitializationMode(myFMU)
# fmiGetReal(myFMU, "u_real")
# fmiGetBoolean(myFMU, "u_boolean")
# fmiGetInteger(myFMU, "u_integer")
# fmiGetString(myFMU, "p_string")
fmiExitInitializationMode(myFMU)

From here on, you may want to simulate the FMU. Please note, that with the default `executionConfig`, it is necessary to prevent a new instantiation using the keyword `instantiate=false`. Otherwise, a new instance is allocated for the simulation-call and the parameters set for the previous instance are not transfered.

In [ ]:
simData = fmiSimulate(myFMU, (tStart, tStop); recordValues=params[1:3], saveat=tSave, 
                        instantiate=false, setup=false)

### Unload FMU

The FMU will be unloaded and all unpacked data on disc will be removed.

In [ ]:
fmiUnload(myFMU)

### Summary

Based on this tutorial it can be seen that there are two different variants to set and get parameters.These examples should make it clear to the user how parameters can also be set with different data types. As a small reminder, the sequence of commands for the manual parameterization of an FMU is summarized again. 

`fmiLoad()` &#8594; `fmiInstantiate!()` &#8594; `fmiSetupExperiment()` &#8594; `fmiSetXXX()` &#8594; `fmiEnterInitializationMode()` &#8594; `fmiGetXXX()` &#8594; `fmiExitInitializationMode()` &#8594; `fmiSimualte()` &#8594; `fmiUnload()`